# Training Job with Encrypted Static Assets

In the [notebook about creating a training job in VPC mode](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker-fundamentals/create-training-job/create_training_job_vpc.ipynb) you learnt how to create a SageMaker training job with network isolation. Network isolation enables you to protect your data and model from being intercepted by cyber pirates. 

![pirate](assets/pirate.jpg)

Another way you can protect your static assets is to encrypt them before moving them from location A to location B. In this notebook, you will walk through a few techniques on that with the help of AWS Key Management Service [(AWS KMS)](https://docs.aws.amazon.com/kms/latest/developerguide/overview.html).

Encryption is a wildly used technology, in addition to the above introductory material, you can find many free lectures online. 

## Symmetric Ciphers
We will focus on symmetric ciphers in this notebook. Quote from the GNU Privacy Handbook

> A symmetric cipher is a cipher that uses the same key for both encryption and decryption. Two parties communicating using a symmetric cipher must agree on the key beforehand. Once they agree, the sender encrypts a message using the key, sends it to the receiver, and the receiver decrypts the message using the key. As an example, the German Enigma is a symmetric cipher, and daily keys were distributed as code books. Each day, a sending or receiving radio operator would consult his copy of the code book to find the day's key. Radio traffic for that day was then encrypted and decrypted using the day's key. Modern examples of symmetric ciphers include 3DES, Blowfish, and IDEA.

## Environment to run this notebook
You can run this notebook on your local machine or EC2 instance as an IAM user or you can run it on SageMaker Notebook Instance as a SageMaker service role. To avoid confusion, we will assume you are running it as an IAM user.

## Permissions
You will need to attach the following permissions to the IAM user

* IAMFullAccess 
* AWSKeyManagementServicePowerUser
* AmazonEC2ContainerRegistryFullAccess

## Outline of this notebook

* Generate a symmetric customer master key (CMK)
* Allow your SageMaker service role to use the CMK
* Generate a data key from the CMK
* Encrypt some data with the data key and upload the encrypted data to S3
* Create a SageMaker service role
* Build a training image 
* Create a SageMaker training job using the encrypted data
* Verify that data retrieved from S3 is encrypted and SageMaker needs your data key to decrypt

The process of using a data key to encrypt your data instead of using master key directly is called [**envelope encryption**](https://docs.aws.amazon.com/kms/latest/developerguide/concepts.html#enveloping)
You can directly use the master key to encrypt your data, but by using a data key, you reduced the risk of [man-in-the-middle-attack](https://en.wikipedia.org/wiki/Man-in-the-middle_attack). 
We will discuss the use of data key in detail later. 

![envelope-encryption](assets/envelope-encryption.jpg)

In [ ]:
# set ups
import boto3
import datetime
import json
import pprint

pp = pprint.PrettyPrinter(indent=1)
kms = boto3.client('kms') 

In [ ]:
# Some helper functions

def current_time():
    ct = datetime.datetime.now() 
    return str(ct.now()).replace(":", "-").replace(" ", "-")[:19]

def account_id():
    return boto3.client('sts').get_caller_identity()['Account']

### Generate a symmetric customer master key

You will use [kms:CreateKey](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/kms.html#KMS.Client.create_key) API to generate a **symmetric customer master key** used for **encryption** and **decryption**. You need to use a IAM policy to define who has access (and with what level of access) to the key. 
If you create the key from AWS console, then by following the default steps you will end up the following key policy:

In [ ]:
root_arn = f"arn:aws:iam::{account_id()}:root"
user_arn = boto3.client('sts').get_caller_identity()['Arn']

key_policy = {
    "Id": "key-consolepolicy-3",
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "Enable IAM User Permissions",
            "Effect": "Allow",
            "Principal": {
                "AWS": root_arn # enable root user to perform all actions
            },
            "Action": "kms:*",
            "Resource": "*"
        },
        
        {
            "Sid": "Allow access for Key Administrators",
            "Effect": "Allow",
            "Principal": {
                "AWS": [user_arn]   # give myself admin permission to this key
                                     # you can add more admin users by appending this list
            },
            "Action": [
                "kms:Create*",
                "kms:Describe*",
                "kms:Enable*",
                "kms:List*",
                "kms:Put*",
                "kms:Update*",
                "kms:Revoke*",
                "kms:Disable*",
                "kms:Get*",
                "kms:Delete*",
                "kms:TagResource",
                "kms:UntagResource",
                "kms:ScheduleKeyDeletion",
                "kms:CancelKeyDeletion"
            ],
            "Resource": "*"
        },
        {
            "Sid": "Allow use of the key",
            "Effect": "Allow",
            "Principal": {
                "AWS": [user_arn]   # allow myself to use the key
                                     # you can add more users / roles to this list
                                     # for example you can add SageMaker service role 
                                     # here. But we will allow SageMaker service role
                                     # to use this key via grant (see below)

            },
            "Action": [
                "kms:Encrypt",
                "kms:Decrypt",
                "kms:ReEncrypt*",
                "kms:GenerateDataKey*",
                "kms:DescribeKey"
            ],
            "Resource": "*"
        },
        {
            "Sid": "Allow attachment of persistent resources",
            "Effect": "Allow",
            "Principal": {
                "AWS": [user_arn] # allow myself to create grant for this key
                                   # see ref below to understand the diff 
                                   # between user and grant
                                   # https://docs.aws.amazon.com/kms/latest/developerguide/concepts.html#grant
            },
            "Action": [
                "kms:CreateGrant",
                "kms:ListGrants",
                "kms:RevokeGrant"
            ],
            "Resource": "*",
            "Condition": {
                "Bool": {
                    "kms:GrantIsForAWSResource": "true"
                }
            }
        }
    ]
}

key_policy = json.dumps(key_policy)

In [ ]:
# create a key with the above key policy

ck_res = kms.create_key(
    Policy=key_policy,
    Description="a symmetric key to demonstrate KMS",
    KeyUsage="ENCRYPT_DECRYPT",                # use this key to encrypt and decrypt
    Origin='AWS_KMS',                          # created via AWS KMS
    CustomerMasterKeySpec='SYMMETRIC_DEFAULT'  # symmetric key
)


pp.pprint(ck_res)

In [ ]:
master_key = ck_res['KeyMetadata']['KeyId']
print("The id of the key: ")
print(master_key)

You can use this master key to encrypt your data directly. It is not a good practice in production. But it is good to know what you can do.

In [ ]:
my_secret_message = "1729 is the smallest number expressible \
as the sum of two cubes in two different ways".encode('utf-8')

# 1729 =  1^3 + 12^3 = 9^3 + 10^3 (Srinivasa Ramanujan)

# make the above secret a ciphertext
enc_res = kms.encrypt(
    KeyId=master_key,
    Plaintext=my_secret_message)

pp.pprint(enc_res)

In [ ]:
# decrypt your secret message
dec_res = kms.decrypt(
    KeyId=master_key,
    CiphertextBlob=enc_res['CiphertextBlob']
)

print("Decrpyted message:")
print(dec_res['Plaintext'].decode())

One thing to notice is encryption and decryption should happen at **bytes** level. If you want to encrypt a python object (list, numpy array, pandas data frame, pytorch model or a string) then the first step is to serialize it into bytes. One easy way to do it is to use `pickle.dumps` method. 